In [2]:
from zipfile import ZipFile
import shutil
import os

apkg_file = "pleco_imported.apkg"
experiement_folder = r"pleco_imported_direct_sqlite_apkg"

if os.path.isdir(experiement_folder):
    shutil.rmtree(experiement_folder)
 

with ZipFile(apkg_file) as zf:
    zf.extractall(experiement_folder)

In [4]:
import sqlite3

In [7]:
con = sqlite3.connect(f"{experiement_folder}\\collection.anki2")

In [8]:
cur = con.cursor()

In [11]:
# The result of a "cursor.execute" can be iterated over by row
for row in list(cur.execute('SELECT * FROM notes;'))[:1]:
    print(row)

(1670388896285, '5|qT2', 1669412005070, 1670426113, 573, ' hsk_level_1 ', '愛\x1f<div align="left"><p><span style="font-size:32px";>&#24859;&#12308;&#29233;&#12309;</span><br>\n<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span><span style="color:#DF80FF;"><span style="font-weight:600;">&#224;i</span></span></p>\n</div><div align="left"><p><b><span style="font-size:0.80em;"><span style="color:#B4B4B4;">NOUN</span></span></b><br>\nlove; affection<br>\n</p>\n<blockquote style="border-left: 2px solid #00aaff; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><span style="color:#00AAFF;">&#27597;</span><span style="color:#00AAFF;"><b>&#24859;</b></span><br>\n<span style="font-weight:600;">m&#468;\'</span><b>&#224;i</b><br>\nmaternal love<br>\n</p>\n</blockquote>\n<blockquote style="border-left: 2px solid #00aaff; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><span style="color:#00AAFF;">&#29238;</s